In [1]:
import time
import copy
import random

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

from graph_tree_module import *


In [2]:
raw = pd.read_csv(r'../Data/train.csv')
# print(raw)

random_seed = 42

# print(list(raw.columns)[:])

In [3]:
data = raw.iloc[:,2:-1]
data = data.fillna(0)
data = data[:2000]
print(data.shape)
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data)

# label
y = raw.iloc[:,-1]
# print(y)

col_name = list(raw.columns)[2:-1]
print(col_name)
# print(normalized_data)

(2000, 14)
['Popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_in min/ms', 'time_signature']


In [4]:
data_min = scaler.data_min_
data_max = scaler.data_max_

# print(data_min, data_max)

In [5]:
def test_query_generator():
    return [random.uniform(data_min[i], data_max[i]) for i in range(len(data_max))]

In [6]:
print(test_query_generator())

[np.float64(24.12112791234021), np.float64(0.9552504528167233), np.float64(0.19056080470002487), np.float64(4.112532450646191), np.float64(-16.524485017835428), np.float64(0.4646192988812883), np.float64(0.8649303837449498), np.float64(0.8947678531547354), np.float64(0.5782854121586177), np.float64(0.09040467987291866), np.float64(0.7063795199427878), np.float64(137.79023701638832), np.float64(699761.9335342847), np.float64(1.8677809711066984)]


In [7]:
import time

class Timer:
    def __enter__(self):
        self.start_time = time.perf_counter()
        return self

    def __exit__(self, *args):
        self.end_time = time.perf_counter()
        self.elapsed_time = self.end_time - self.start_time
        print(f"Elapsed time: {self.elapsed_time} seconds")

In [8]:
test_tree = graph_tree(attr_name = col_name, X = normalized_data, X_maxmin=[scaler.data_max_,scaler.data_min_], min_cluster_size=200)
# test_tree.show_root()
with Timer():
    test_tree.build_tree()

root node is created
[0.44036364 0.5291267  0.65660264 0.47645455 0.76121848 0.6405
 0.06197007 0.25581723 0.13625976 0.18137536 0.48564998 0.48213134
 0.18333419 0.728875  ] [0.44036364 0.5291267  0.65660264 0.47645455 0.76121848 0.6405
 0.06197007 0.25581723 0.13625976 0.18137536 0.48564998 0.48213134
 0.18333419 0.728875  ] [[<graph_tree_module.Node object at 0x000001724B54B560>, <graph_tree_module.Node object at 0x000001724B54B560>], [<graph_tree_module.Node object at 0x000001724B54B560>, <graph_tree_module.Node object at 0x000001724B54B560>], [<graph_tree_module.Node object at 0x000001724B54B560>, <graph_tree_module.Node object at 0x000001724B54B560>], [<graph_tree_module.Node object at 0x000001724B54B560>, <graph_tree_module.Node object at 0x000001724B54B560>], [<graph_tree_module.Node object at 0x000001724B54B560>, <graph_tree_module.Node object at 0x000001724B54B560>], [<graph_tree_module.Node object at 0x000001724B54B560>, <graph_tree_module.Node object at 0x000001724B54B560>]

In [9]:
# import pickle

# with open("graph_tree.pkl", 'wb') as f:
#     pickle.dump(test_tree, f)

In [10]:

seaching = test_query_generator()
with Timer():
    cur = test_tree.search(seaching)

Elapsed time: 0.0004523999959928915 seconds


In [11]:
print(cur.id)

8


In [12]:
a = test_tree._get_closest_node_within_lv(cur, seaching, mode="lv", lv=2)
print(a)

AttributeError: 'list' object has no attribute 'down_lv'

In [53]:
a

{'cur_cluster': <graph_tree_module.Node at 0x13d0120f230>,
 'super_cluster': <graph_tree_module.Node at 0x13d0120d700>,
 'neibour_cluster': [<graph_tree_module.Node at 0x13d0120d250>,
 'sub_cluster': None,
 'contra_cluster': [<graph_tree_module.Node at 0x13d5bf98f80>,
  <graph_tree_module.Node at 0x13d01201fd0>]}

In [54]:
test_tree.print_tree(simple=0)

---------- lv: 0, node num.: 1 ----------
0
---------- lv: 1, node num.: 2 ----------
2
1
---------- lv: 2, node num.: 6 ----------
4
3
6
5
7
8
---------- lv: 3, node num.: 10 ----------
10
9
11
13
12
16
14
15
17
18
---------- lv: 4, node num.: 6 ----------
19
20
22
21
24
23
---------- lv: 5, node num.: 5 ----------
25
27
26
28
29


In [55]:
res = test_tree.reflection_search(np.mean(normalized_data[:100], axis=0), test_tree.root)

In [56]:
res

 [np.int64(934),
  np.int64(1807),
  np.int64(461),
  np.int64(1199),
  np.int64(1193),
  np.int64(1251),
  np.int64(1318),
  np.int64(381),
  np.int64(1006),
  np.int64(107)]]

In [57]:
print(normalized_data.shape)
print(np.mean(normalized_data[:100], axis=0))

print(test_tree.root.center)

(2000, 14)
[0.42919192 0.53485346 0.62770415 0.45818182 0.7529106  0.72
 0.05419521 0.24029204 0.1354953  0.16829518 0.4967118  0.46979964
 0.17562644 0.72      ]
[0.44036364 0.5291267  0.65660264 0.47645455 0.76121848 0.6405
 0.06197007 0.25581723 0.13625976 0.18137536 0.48564998 0.48213134
 0.18333419 0.728875  ]


In [58]:
from collections import Counter

"""
create a new index (something like tf-idf)
data reduction rate
vs
accurate
"""
def score(search_data, acc, smoothing = 0.001, lam = 0.5):
    return lam*(acc/100)+(1-lam)*(1-(search_data/(1000+smoothing)))

my_score = []
normal_kmeans_score = []

tmp = [i*100 for i in range(10)]
for i in tmp:
    b = i
    u = i+100
    # print(normalized_data[b:u])
    print(f"-- mine result{i}")
    seaching = np.mean(normalized_data[b:u], axis=0)
    result = test_tree.search(seaching)
    
    print("search number: ", len(result.index))
    right = np.where((result.index>=b) & (result.index<u))[0] # return index
    print("hitted right result:", len(right))
    my_score.append(score(len(result.index), len(right)))

    ############
    print("-- kmeans result")
    checking = KMeans(n_clusters=10, random_state=0).fit(normalized_data)

    counts1 = Counter(checking.labels_)
    print(counts1)

    counts2 = Counter(checking.labels_[b:u])
    print(counts2)
    belongs_group = counts2.most_common()[0][0]
    print("belongs to group: ", belongs_group, "\n")
    # print(counts1.get(belongs_group), counts2.get(belongs_group))
    normal_kmeans_score.append(score(counts1.get(belongs_group), counts2.get(belongs_group)))

    

-- mine result0
search number:  2000
hitted right result: 100
-- kmeans result
Counter({np.int32(4): 410, np.int32(2): 271, np.int32(6): 261, np.int32(1): 249, np.int32(8): 224, np.int32(0): 158, np.int32(5): 145, np.int32(7): 115, np.int32(3): 90, np.int32(9): 77})
Counter({np.int32(2): 20, np.int32(4): 20, np.int32(6): 14, np.int32(1): 12, np.int32(8): 9, np.int32(5): 7, np.int32(0): 6, np.int32(3): 6, np.int32(7): 5, np.int32(9): 1})
belongs to group:  2 

-- mine result100
search number:  2000
hitted right result: 100
-- kmeans result
Counter({np.int32(4): 410, np.int32(2): 271, np.int32(6): 261, np.int32(1): 249, np.int32(8): 224, np.int32(0): 158, np.int32(5): 145, np.int32(7): 115, np.int32(3): 90, np.int32(9): 77})
Counter({np.int32(4): 22, np.int32(6): 11, np.int32(1): 11, np.int32(2): 11, np.int32(5): 10, np.int32(0): 10, np.int32(7): 8, np.int32(9): 7, np.int32(8): 7, np.int32(3): 3})
belongs to group:  4 

-- mine result200
search number:  2000
hitted right result: 100
-- k

In [59]:
print(my_score)
print(normal_kmeans_score)

print(sum(my_score))
print(sum(normal_kmeans_score))

[9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07]
[0.46450013549986446, 0.40500020499979494, 0.37000020499979497, 0.41500020499979495, 0.40000020499979494, 0.43000020499979497, 0.3850002049997949, 0.41500020499979495, 0.42500020499979496, 0.4645001304998695]
9.99998999939855e-06
4.174001905998094
